
## **SISTEMA DE RECOMENDACIÓN Y PREDICCIÓN DE PELÍCULAS POR CONTENIDO Y VALORACIONES DE USUARIO**



---



Para el desarrollo del trabajo, se optó por hacer uso de un dataset extraído de internet (url: https://www.kaggle.com/shivamb/netflix-shows) llamado *netflix_titles.csv*, el cual contiene películas y series de televisión de la plataforma de streaming Netflix. 

Este archivo .csv lo incluimos dentro de la carpeta content para poder trabajar con él.




## Gestión CRUD



---





**CARGA DEL DATASET Y FILTRADO**

El dataset *netflix_titles.csv*, que se había guardado dentro de la carpeta sample_data, se carga y se guarda dentro de la variable originalData.


In [1]:
import pandas as pd

originalData = pd.read_csv('/content/netflix_titles.csv')
originalData.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
5,80163890,TV Show,Apaches,NaN,"Alberto Ammann, Eloy Azorín, Verónica Echegui,...",Spain,"September 8, 2017",2016,TV-MA,1 Season,"Crime TV Shows, International TV Shows, Spanis...",A young journalist is forced into a life of cr...
6,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada","September 8, 2017",2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
7,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,"September 8, 2017",2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
8,80117902,TV Show,Fire Chasers,NaN,NaN,United States,"September 8, 2017",2017,TV-MA,1 Season,"Docuseries, Science & Nature TV","As California's 2016 fire season rages, brave ..."
9,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden","September 8, 2017",2014,R,90 min,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...



Dado que el proyecto está enfocado a la recomendación de películas y el dataset contiene tanto películas como series de televisión, se debe realizar un filtrado sobre éste. Para ello, nos quedamos con aquellas filas que tengan como valor 'Movie' en la columna 'type'.

In [2]:
filteredMovies = (originalData["type"] == "Movie")
dataMovies = originalData[filteredMovies]
dataMovies.reset_index().head(10)

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
3,6,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada","September 8, 2017",2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,7,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,"September 8, 2017",2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
5,9,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden","September 8, 2017",2014,R,90 min,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...
6,10,80169755,Movie,Joaquín Reyes: Una y no más,José Miguel Contreras,Joaquín Reyes,NaN,"September 8, 2017",2017,TV-MA,78 min,Stand-Up Comedy,Comedian and celebrity impersonator Joaquín Re...
7,11,70299204,Movie,Kidnapping Mr. Heineken,Daniel Alfredson,"Jim Sturgess, Sam Worthington, Ryan Kwanten, A...","Netherlands, Belgium, United Kingdom, United S...","September 8, 2017",2015,R,95 min,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ..."
8,12,80182480,Movie,Krish Trish and Baltiboy,NaN,"Damandeep Singh Baggan, Smita Malhotra, Baba S...",NaN,"September 8, 2017",2009,TV-Y7,58 min,Children & Family Movies,"A team of minstrels, including a monkey, cat a..."
9,13,80182483,Movie,Krish Trish and Baltiboy: Battle of Wits,"Munjal Shroff, Tilak Shetty","Damandeep Singh Baggan, Smita Malhotra, Baba S...",NaN,"September 8, 2017",2013,TV-Y7,62 min,Children & Family Movies,"An artisan is cheated of his payment, a lion o..."



Por otra parte, se optó por trabajar únicamente con las columnas show_id(id de la película), title (título de la película), listed_in (géneros de la película) y description (descripción de la película). Para ello, se eliminaron todas las columnas que no fueran a ser utilizadas. Finalmente, se renombraron los índices de las filas para que se adaptasen al nuevo dataset *dataMovies*

In [3]:
dataMovies = dataMovies.drop(columns=['type','director','cast','country','date_added','release_year','rating','duration'])
dataMovies = dataMovies.reset_index()
dataMovies = dataMovies.drop(columns=['index'])
dataMovies.head(10)

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
5,70304990,Good People,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...
6,80169755,Joaquín Reyes: Una y no más,Stand-Up Comedy,Comedian and celebrity impersonator Joaquín Re...
7,70299204,Kidnapping Mr. Heineken,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ..."
8,80182480,Krish Trish and Baltiboy,Children & Family Movies,"A team of minstrels, including a monkey, cat a..."
9,80182483,Krish Trish and Baltiboy: Battle of Wits,Children & Family Movies,"An artisan is cheated of his payment, a lion o..."



El dataset ya filtrado se guarda dentro de la carpeta sample_data como dataMovies.csv

In [4]:
dataMovies.to_csv('dataMovies.csv')
print(dataMovies.head(10))

    show_id  ...                                        description
0  81145628  ...  Before planning an awesome wedding for his gra...
1  80117401  ...  Jandino Asporaat riffs on the challenges of ra...
2  80125979  ...  When nerdy high schooler Dani finally attracts...
3  70304989  ...  In a dystopian future, an insurance adjuster f...
4  80164077  ...  Fabrizio Copano takes audience participation t...
5  70304990  ...  A struggling couple can't believe their luck w...
6  80169755  ...  Comedian and celebrity impersonator Joaquín Re...
7  70299204  ...  When beer magnate Alfred "Freddy" Heineken is ...
8  80182480  ...  A team of minstrels, including a monkey, cat a...
9  80182483  ...  An artisan is cheated of his payment, a lion o...

[10 rows x 4 columns]




---


**PROCESADO MEDIANTE LA DESCRIPCIÓN Y GÉNERO DE LAS PELÍCULAS**

Para la recomendación de películas, se ha optado por tener en cuenta tanto la descripción de las películas como el género de éstas. Para ello, se calculan dos matrices de distancias distintas. La matriz de distancias de la descripción textual guarda valores entre 0 y 1. La segunda, guarda distancias entre 0 y 0,125. 
Para el cálculo de distancias final se suman ambas matrices obteniendo así valores entre 0 y 1.125. 

Inicialmente, cargamos el archivo .csv que habíamos guardado previamente y lo guardamos dentro de la variable originalData. Todo este proceso se hace a través de la función LoadData().




In [15]:
import pandas as pd

def LoadData():
  originalData = pd.read_csv('/content/dataMovies.csv')
  originalData = originalData.drop(columns=['Unnamed: 0'])
  return originalData

originalData = LoadData()
originalData.head(10)

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
5,70304990,Good People,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...
6,80169755,Joaquín Reyes: Una y no más,Stand-Up Comedy,Comedian and celebrity impersonator Joaquín Re...
7,70299204,Kidnapping Mr. Heineken,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ..."
8,80182480,Krish Trish and Baltiboy,Children & Family Movies,"A team of minstrels, including a monkey, cat a..."
9,80182483,Krish Trish and Baltiboy: Battle of Wits,Children & Family Movies,"An artisan is cheated of his payment, a lion o..."


Se obtiene el dataset preprocessedData, que contiene una nueva columna llamada processed_text en la que se guardan las palabras más relevantes de la descripción para la creación de una bolsa de palabras. Todo este proceso está contenido dentro de la función ProcessedText().

In [16]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

def ProcessedText():
  ps = PorterStemmer()

  preprocessedText = []

  for row in originalData.itertuples():
    text = word_tokenize(row[4]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    preprocessedText.append(text)

  preprocessedData = originalData
  preprocessedData['processed_text'] = preprocessedText
  return preprocessedData

preprocessedData = ProcessedText()
preprocessedData.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,show_id,title,listed_in,description,processed_text
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,befor plan awesom wed grandfath polar bear kin...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,jandino asporaat riff challeng rais kid serena...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...,when nerdi high schooler dani final attract in...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",In dystopian futur insur adjust tech compani i...
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,fabrizio copano take audienc particip next lev...
5,70304990,Good People,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,A struggl coupl ca believ luck find stash mone...
6,80169755,Joaquín Reyes: Una y no más,Stand-Up Comedy,Comedian and celebrity impersonator Joaquín Re...,comedian celebr imperson joaquín rey decid zes...
7,70299204,Kidnapping Mr. Heineken,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ...",when beer magnat alfr freddi heineken kidnap 1...
8,80182480,Krish Trish and Baltiboy,Children & Family Movies,"A team of minstrels, including a monkey, cat a...",A team minstrel includ monkey cat donkey narra...
9,80182483,Krish Trish and Baltiboy: Battle of Wits,Children & Family Movies,"An artisan is cheated of his payment, a lion o...",An artisan cheat payment lion throne brother i...


A partir de preprocessedData y su campo processed_text se crea una 'prematriz'.
Esta prematriz estará compuesta por textsBows que se crean haciendo uso de la librería. Estos textsBows almacenan la información de las frecuencias relativas de cada palabra. Todo este proceso se guarda dentro de la función TextsBow().

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

def TextsBoW():
  bagOfWordsModel = TfidfVectorizer()
  bagOfWordsModel.fit(preprocessedData['processed_text'])
  textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
  return textsBoW

textsBoW = TextsBoW()
print("Finished")

Finished


In [18]:
print(textsBoW[10])

  (0, 8420)	0.1835787545856911
  (0, 6973)	0.28129166024763186
  (0, 6827)	0.3178209314072413
  (0, 5890)	0.3292897982928149
  (0, 5734)	0.26787770205547956
  (0, 5589)	0.28549201141576924
  (0, 4274)	0.22132941337485013
  (0, 4273)	0.20096460221097676
  (0, 3393)	0.23369419355147228
  (0, 3283)	0.3292897982928149
  (0, 2516)	0.29551102893678927
  (0, 1468)	0.2557032156227651
  (0, 987)	0.34545425828855103



Se calculan las distancias a través de un algoritmo. Se optó por hacer uso de la *Similitud Coseno* para obtener las distancias entre **vectores**.

Todo este proceso se hace dentro de la función Distance().

In [19]:
from sklearn.metrics import pairwise_distances

def Distance():
  distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')
  return distance_matrix

distance_matrix = Distance()


Con la prematriz con las frecuencias relativas crearemos una matriz NxN, donde N es el número de películas.
Cada celda devuelve la distancia entre las películas que representan cada índice, es decir i, j.

In [20]:
print(distance_matrix.shape)
print(type(distance_matrix))

(4265, 4265)
<class 'numpy.ndarray'>


Una vez calculadas las distancias en base a las descripciones de las películas, se deben calcular las distancias teniendo en cuenta el género.

Se obtiene el dataset preprocessedData, que contiene una nueva columna llamada processed_genre en la que se guardan las palabras más relevantes del género para la creación de una bolsa de palabras. Todo este proceso está contenido dentro de la función ProcessedGenre().

In [21]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

def ProcessedGenre():
  ps = PorterStemmer()

  preprocessedGenre = []

  for row in originalData.itertuples():
    text = word_tokenize(row[3]) ## indice de la columna que contiene el género
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    preprocessedGenre.append(text)

  preprocessedData = originalData
  preprocessedData['processed_genre'] = preprocessedGenre
  return preprocessedData

preprocessedData = ProcessedGenre()
preprocessedData.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,show_id,title,listed_in,description,processed_text,processed_genre
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,befor plan awesom wed grandfath polar bear kin...,children famili movi comedi
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,jandino asporaat riff challeng rais kid serena...,comedi
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...,when nerdi high schooler dani final attract in...,comedi
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",In dystopian futur insur adjust tech compani i...,intern movi fantasi thriller
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,fabrizio copano take audienc particip next lev...,comedi
5,70304990,Good People,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,A struggl coupl ca believ luck find stash mone...,action adventur thriller
6,80169755,Joaquín Reyes: Una y no más,Stand-Up Comedy,Comedian and celebrity impersonator Joaquín Re...,comedian celebr imperson joaquín rey decid zes...,comedi
7,70299204,Kidnapping Mr. Heineken,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ...",when beer magnat alfr freddi heineken kidnap 1...,action adventur drama intern movi
8,80182480,Krish Trish and Baltiboy,Children & Family Movies,"A team of minstrels, including a monkey, cat a...",A team minstrel includ monkey cat donkey narra...,children famili movi
9,80182483,Krish Trish and Baltiboy: Battle of Wits,Children & Family Movies,"An artisan is cheated of his payment, a lion o...",An artisan cheat payment lion throne brother i...,children famili movi


A partir de preprocessedData y su campo processed_genre se crea una 'prematriz'.
Esta prematriz estará compuesta por textsBows que se crean haciendo uso de la librería. Estos textsBows almacenan la información de las frecuencias relativas de cada palabra. Todo este proceso se guarda dentro de la función TextsBowG().

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

def TextsBoWG():
  bagOfWordsModel = TfidfVectorizer()
  bagOfWordsModel.fit(preprocessedData['processed_genre'])
  TextsBoWG= bagOfWordsModel.transform(preprocessedData['processed_genre'])
  return TextsBoWG

textsBoWG = TextsBoWG()
print("Finished")

Finished


In [23]:
print(textsBoWG[10])

  (0, 17)	0.2644716934023223
  (0, 10)	0.6819291471219383
  (0, 3)	0.6819291471219383


Se calculan las distancias a través de un algoritmo. Se optó por hacer uso de la *Similitud Coseno* para obtener las distancias entre **vectores**. Cabe destacar que las distancias se dividen entre 8 para que el género no tenga tanto peso como las descripciones de las películas. Todo este proceso se hace dentro de la función DistanceG().

In [24]:
from sklearn.metrics import pairwise_distances

def DistanceG():
  distance_matrixG= pairwise_distances(textsBoWG,textsBoWG,metric='cosine')/8
  return distance_matrixG

distance_matrixG = DistanceG()

Con la prematriz con las frecuencias relativas crearemos una matriz NxN, donde N es el número de películas.
Cada celda devuelve la distancia entre las películas que representan cada índice, es decir i, j.

In [25]:
print(distance_matrixG.shape)
print(type(distance_matrixG))

(4265, 4265)
<class 'numpy.ndarray'>


Una vez calculadas las dos matrices de distancias, solo hace falta sumarlas entre sí para obtener una matriz de distancias final que tenga en cuenta tanto la descripción textual de las películas como el género de estas.

In [26]:
def add(distance_matrix, distance_matrixG):
    for i in range(distance_matrix.shape[0]):
        for j in range(distance_matrix.shape[1]):
            distance_matrix[i][j] = distance_matrix[i][j] + distance_matrixG[i][j]
    return distance_matrix

distance_matrixFinal = add(distance_matrix,distance_matrixG)
print(distance_matrixFinal[0:11])

[[0.         1.07638423 1.07638423 ... 1.11741137 1.125      1.09454381]
 [1.07638423 0.         1.         ... 1.125      1.125      1.125     ]
 [1.07638423 1.         0.         ... 1.125      1.125      1.125     ]
 ...
 [1.00984138 1.125      1.125      ... 1.11676285 1.125      1.09194104]
 [1.00984138 1.125      1.125      ... 1.11676285 1.125      1.09194104]
 [1.00984138 1.125      1.125      ... 1.11676285 1.125      1.09194104]]




---


**AÑADIR PELÍCULA**

El añadido de una película se divide en tres partes. En la primera, se inserta la nueva película en el dataset de películas. En la segunda, se vuelve a realizar el cálculo de distancias para que esta nueva película sea tenida en cuenta a la hora de recomendar. En la última, se almacena la nueva tabla generada con la película insertada.

Ha de tenerse en cuenta que para que todo el código que viene a continuación funcione, deben haberse ejecutado todos los bloques de código del apartado **"Procesado mediante la descripción de las películas"** de la sección Gestión CRUD. Esto es debido a que en dicha sección se definen varias funciones que serán usadas en ésta.



**1.Agregación de la Película al dataset**

Primero de todo, debe cargarse el dataset que contiene todas las películas.



In [ ]:
import pandas as pd

originalData = LoadData()
originalData.tail(10)

,show_id,title,listed_in,description
4255,80100054,Skins,"Dramas, International Movies","Deformed, disfigured characters must find a wa..."
4256,80097468,The Bad Kids,Documentaries,"In this documentary, teachers at a Mojave Dese..."
4257,80171439,The Human Factor: The Untold Story of the Bomb...,"Documentaries, International Movies, Music & M...",A family of Parsi musicians collectively works...
4258,80104237,The Tenth Man,"Dramas, International Movies",After spending much of his adult life in New Y...
4259,80093107,Toro,"Dramas, International Movies, Thrillers",Ex-con Toro's brother and former partner in cr...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...
4264,80116008,Little Baby Bum: Nursery Rhyme Friends,Movies,Nursery rhymes and original music for children...



Una vez cargado el dataset, se añade la nueva fila y se comprueba que se haya agregado correctamente al dataset.

In [ ]:
codigo = hash('Alberto') % 100000000
existe = False

while existe == False:
  if (originalData['show_id'] == codigo).any():
    codigo = (codigo + 1) % 100000000
  else:
    existe = True

originalData = originalData.append({'show_id' : codigo , 'title' : 'Alberto', 'listed_in' : 'Comedy', 'description' : 'This film is for testing'} , ignore_index=True)

filtered = (originalData["show_id"] == codigo)
comprobacion = originalData[filtered]
comprobacion

,show_id,title,listed_in,description
4265,38609190,Alberto,Comedy,This film is for testing


**2.Cálculo de distancias**


Ahora se debe actualizar la bolsa de palabras y realizar nuevamente el cálculo de distancias para que se tenga en cuenta la nueva película insertada.


In [ ]:
preprocessedData = ProcessedText()
textsBoW = TextsBoW()
distance_matrix = Distance()

preprocessedData = ProcessedGenre()
textsBoWG = TextsBoWG()
distance_matrixG = DistanceG()

distance_matrixFinal = add(distance_matrix,distance_matrixG)

print(distance_matrixFinal.shape)
print(type(distance_matrixFinal))

(4266, 4266)
<class 'numpy.ndarray'>



A continuación, se incluye un ejemplo de cómo el sistema es capaz de recomendar películas semejantes a la nueva película agragada.

In [ ]:
searchTitle = "Alberto" #Película base para las recomendaciones
indexOfTitle = preprocessedData[preprocessedData['title']==searchTitle].index.values[0]
distance_scores = list(enumerate(distance_matrixFinal[indexOfTitle]))
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
top_scores = ordered_scores[0:11]
print(top_scores)
top_indexes = [i[0] for i in top_scores]
print(preprocessedData['title'].iloc[top_indexes])

[(4265, 0.0), (3554, 0.8007530757456257), (1800, 0.8512691067126563), (3719, 0.8662636862217409), (3510, 0.8792849906745047), (2520, 0.8904360274431539), (2308, 0.8940864776123613), (1533, 0.8945152280359095), (45, 0.8952050250142475), (2528, 0.8952890264125454), (444, 0.8969661556192621)]
4265                                          Alberto
3554                  Martin Lawrence Live: Runteldat
1800                                         No Entry
3719                                    One Last Shot
3510                                A Trip to Jamaica
2520                                  Mi Obra Maestra
2308                                Chris Tucker Live
1533    Undercover: How to Operate Behind Enemy Lines
45                               Marc Maron: Too Real
2528                                The Good Catholic
444          Monty Python: Live at The Hollywood Bowl
Name: title, dtype: object



**3.Almacenamiento del Dataset**


Finalmente, se guarda el dataset con la nueva película agregada.

In [ ]:
originalData.to_csv('originalData.csv')
print(originalData.tail(10))

       show_id  ...                      processed_genre
4256  80097468  ...                          documentari
4257  80171439  ...  documentari intern movi music music
4258  80104237  ...                    drama intern movi
4259  80093107  ...           drama intern movi thriller
4260  80085438  ...                          documentari
4261  80085439  ...           comedi drama independ movi
4262  80011846  ...               documentari sport movi
4263  80064521  ...                          documentari
4264  80116008  ...                                 movi
4265  38609190  ...                               comedi

[10 rows x 6 columns]




---


**MODIFICAR PELÍCULA**

Al igual que pasaba en *Añadir Película*, la modificación de una película también se divide en tres partes: modificación de la película; actualización de las distancias; y guardado del dataset modificado.


Ha de tenerse en cuenta que para que todo el código que viene a continuación funcione, deben haberse ejecutado todos los bloques de código de la sacción **"Procesado mediante la descripción de las películas"**. Esto es debido a que en dicha sección se definen varias funciones que serán usadas en ésta.

**1.Modificación de la Película en el dataset**

Primero de todo, debe cargarse el dataset que contiene todas las películas.

In [ ]:
import pandas as pd

originalData = LoadData()
originalData.tail(10)

,show_id,title,listed_in,description
4255,80100054,Skins,"Dramas, International Movies","Deformed, disfigured characters must find a wa..."
4256,80097468,The Bad Kids,Documentaries,"In this documentary, teachers at a Mojave Dese..."
4257,80171439,The Human Factor: The Untold Story of the Bomb...,"Documentaries, International Movies, Music & M...",A family of Parsi musicians collectively works...
4258,80104237,The Tenth Man,"Dramas, International Movies",After spending much of his adult life in New Y...
4259,80093107,Toro,"Dramas, International Movies, Thrillers",Ex-con Toro's brother and former partner in cr...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...
4264,80116008,Little Baby Bum: Nursery Rhyme Friends,Movies,Nursery rhymes and original music for children...



Se busca la película a editar y se modifican sus datos.

In [ ]:
codigo = 80117401
existe = False

originalData.loc[originalData['show_id'] == codigo, ['description']] = 'The description\'s value changed '
originalData.head(10)

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,The description's value changed
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
5,70304990,Good People,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...
6,80169755,Joaquín Reyes: Una y no más,Stand-Up Comedy,Comedian and celebrity impersonator Joaquín Re...
7,70299204,Kidnapping Mr. Heineken,"Action & Adventure, Dramas, International Movies","When beer magnate Alfred ""Freddy"" Heineken is ..."
8,80182480,Krish Trish and Baltiboy,Children & Family Movies,"A team of minstrels, including a monkey, cat a..."
9,80182483,Krish Trish and Baltiboy: Battle of Wits,Children & Family Movies,"An artisan is cheated of his payment, a lion o..."


**2.Cálculo de distancias**

Ahora se debe actualizar la bolsa de palabras para que se tenga en cuenta la modificación realizada.

In [ ]:
preprocessedData = ProcessedText()
textsBoW = TextsBoW()
distance_matrix = Distance()

preprocessedData = ProcessedGenre()
textsBoWG = TextsBoWG()
distance_matrixG = DistanceG()

distance_matrixFinal = add(distance_matrix,distance_matrixG)

print(distance_matrixFinal.shape)
print(type(distance_matrixFinal))

(4265, 4265)
<class 'numpy.ndarray'>


**3.Almacenamiento del Dataset**

Finalmente, se guarda en el dataset las modificaciones agregadas a la película.

In [ ]:
originalData.to_csv('originalData.csv')
print(originalData.head(10))

    show_id  ...                    processed_genre
0  81145628  ...        children famili movi comedi
1  80117401  ...                             comedi
2  80125979  ...                             comedi
3  70304989  ...       intern movi fantasi thriller
4  80164077  ...                             comedi
5  70304990  ...           action adventur thriller
6  80169755  ...                             comedi
7  70299204  ...  action adventur drama intern movi
8  80182480  ...               children famili movi
9  80182483  ...               children famili movi

[10 rows x 6 columns]


## Ejemplo de recomendación de películas


---


A continuación, se muestra un ejemplo de uso del sistema de Recomendación. Se seleccinó la película *Automata* para obtener una lista de recomendaciones de películas que guardan cierta similitud con ésta.

Ha de tenerse en cuenta que para que todo el código que viene a continuación funcione, deben haberse ejecutado todos los bloques de código del apartado **"Procesado mediante la descripción de las películas"** de la sección Gestión CRUD. Esto es debido a que es en este apartado donde se genera la matriz de distancias entre películas necesaria para realizar el proceso de recomendación.

In [ ]:
searchTitle = "Automata" #Película base para las recomendaciones
indexOfTitle = preprocessedData[preprocessedData['title']==searchTitle].index.values[0]
indexOfTitle

3


Distancia entre la película Automata con el resto de la matriz.
1.5 -> Ningún parecido.
0.0 -> Completamente igual.

In [ ]:
distance_scores = list(enumerate(distance_matrixFinal[indexOfTitle]))
distance_scores[0:11]

[(0, 1.1179838769274695),
 (1, 1.125),
 (2, 1.125),
 (3, 0.0),
 (4, 1.125),
 (5, 1.078857398522215),
 (6, 1.125),
 (7, 1.10397572808732),
 (8, 1.1173842834543346),
 (9, 1.1173842834543346),
 (10, 1.1173842834543346)]



Ordenadas las distancias. La primera es la propia película Automata, por eso su diferencia es 0.0.

In [ ]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
ordered_scores[0:11]

[(3, 0.0),
 (3128, 0.8803039037690801),
 (2176, 0.8986352017001517),
 (1200, 0.9042604560361982),
 (886, 0.9063044750703562),
 (2566, 0.9112390175427516),
 (1858, 0.9152425359825711),
 (3606, 0.9217303806387209),
 (1923, 0.9267826461474766),
 (3471, 0.9438910722036926),
 (2425, 0.9439620705159506)]


A continuación, se muestran las 10 películas más parecidas a Automata.

In [ ]:
top_scores = ordered_scores[0:11]
top_scores

[(3, 0.0),
 (3128, 0.8803039037690801),
 (2176, 0.8986352017001517),
 (1200, 0.9042604560361982),
 (886, 0.9063044750703562),
 (2566, 0.9112390175427516),
 (1858, 0.9152425359825711),
 (3606, 0.9217303806387209),
 (1923, 0.9267826461474766),
 (3471, 0.9438910722036926),
 (2425, 0.9439620705159506)]

In [ ]:
top_indexes = [i[0] for i in top_scores]
top_indexes

[3, 3128, 2176, 1200, 886, 2566, 1858, 3606, 1923, 3471, 2425]


Títulos de las 10 películas más parecidas.

In [ ]:
preprocessedData['title'].iloc[top_indexes]

3                             Automata
3128                            Master
2176                        Ex Machina
1200                              Anon
886                      The Rainmaker
2566    Roger Corman's Death Race 2050
1858                       Equilibrium
3606                        The Worthy
1923                               TAU
3471                        F.R.E.D.I.
2425                Suicide (Hitabdut)
Name: title, dtype: object

In [ ]:
originalData.iloc[top_indexes]

,show_id,title,listed_in,description,processed_text,processed_genre
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",In dystopian futur insur adjust tech compani i...,intern movi fantasi thriller
3128,80163352,Master,"Action & Adventure, International Movies",Needing hard evidence to convict a company cha...,need hard evid convict compani chairman fraud ...,action adventur intern movi
2176,80023689,Ex Machina,"Dramas, Independent Movies, International Movies",A coder at a tech company wins a week-long ret...,A coder tech compani win retreat compound comp...,drama independ movi intern movi
1200,80195964,Anon,"Dramas, Sci-Fi & Fantasy, Thrillers",In a future where technology has rendered priv...,In futur technolog render privaci obsolet dete...,drama fantasi thriller
886,1181661,The Rainmaker,"Dramas, Thrillers",A young attorney and a scrappy paralegal work ...,A young attorney scrappi paraleg work help par...,drama thriller
2566,80152003,Roger Corman's Death Race 2050,"Action & Adventure, Sci-Fi & Fantasy","In this dystopian sequel, kills equal points i...",In dystopian sequel kill equal point violent g...,action adventur fantasi
1858,60024935,Equilibrium,"Action & Adventure, Dramas, Sci-Fi & Fantasy","In a dystopian future, a totalitarian regime m...",In dystopian futur totalitarian regim maintain...,action adventur drama fantasi
3606,80191513,The Worthy,"International Movies, Sci-Fi & Fantasy, Thrillers","In the near future, with civilization a thing ...",In near futur civil thing past man put famili ...,intern movi fantasi thriller
1923,80217569,TAU,"Sci-Fi & Fantasy, Thrillers",Kidnapped by an inventor who uses her as a tes...,kidnap inventor use test subject perfect robot...,fantasi thriller
3471,81025946,F.R.E.D.I.,"Children & Family Movies, Sci-Fi & Fantasy","Venturing into the woods, high schooler James ...",ventur wood high schooler jame discov intellig...,children famili movi fantasi
